<img src="./img/uomlogo.png" align="left"/><br><br>
# PHYS20762 - Decorators

Hywel Owen  
(c) University of Manchester  
28th May 2020

![](./img/bee.png)
## Modifying Functions Using Decorators

Decorators are an important part of **Python** programming, and is a way of making a *function transformation* whilst still keeping the function readable. By using decorator syntax we make our code much more readable than it would be otherwise - working with our code is therefore 'sweeter', and so this way of writing things is called *syntactic sugar*. (Yes, really.)

![](./img/bee.png)
## Python Functions are First-Class Objects

A very important fact about Python is that functions in Python are *first-class objects*. Just like any other objects, this means that functions can be:  
- be passed to functions as arguments  
- be returned from functions  
- be assigned to variables  

### Assigning functions to other variables

First, we should realise that functions can be defined and then assigned to other variables:

In [1]:
def square_it(x): # Define a function that squares the argument
    return x ** 2

power_it = square_it # Assign this function to another variable (notice there's no argument brackets). 
# power_it now dose the same thing as square_it


print(square_it(4))
print(power_it(4))

16
16


Great, so we can pass functions to other variables. What use is that? Keep reading below...

### Passing functions as arguments

We can pass functions as arguments. This is easiest to understand through an example:

In [3]:
def bark(name):
    print(f'{name} barks!')
    
def sit(name):
    print(f'{name} sits!')

def roll_over(name):
    print(f'{name} rolls over!')
    
def command_dog(name,command): # Make the dog called <name> do command <command>
    command(name)

command_dog("Rover",sit)
command_dog("Rover",bark)
command_dog("Rover",roll_over)


Rover sits!
Rover barks!
Rover rolls over!


But what if we want to modify the functions to include some extra actions? Let's say we want to include some print statements:
- The dog heard the command
- The dog is waiting for the new command  

and we want the same behaviour to happen in all 3 functions.

This would be cumbersome to write 3 times, and would be prone to error (you have to write the modification 3 times!).

We can get around that using **decorators**. Again, this is best demonstrated with an example:

In [6]:
# Define the decorator function
def add_dog_status(fun):
    def wrapper(*args, **kwargs): # Allow arguments to be passed to the original function
        print(f'The dog has heard the command {fun.__name__}.') # Add status before
        res = fun(*args, **kwargs) # Get any return values (there aren't any here)
        print(f'The dog has finished the command {fun.__name__}.') # Add status after
        return res # Return any return values
        
    return wrapper

@add_dog_status
def bark(name):
    print(f'{name} barks!')

@add_dog_status
def sit(name):
    print(f'{name} sits!')

@add_dog_status
def roll_over(name):
    print(f'{name} rolls over!')
    
def command_dog(name,command): # Make the dog called <name> do command <command>
    command(name)

command_dog("Rover",sit)
command_dog("Rover",bark)
command_dog("Rover",roll_over)

The dog has heard the command sit.
Rover sits!
The dog has finished the command sit.
The dog has heard the command bark.
Rover barks!
The dog has finished the command bark.
The dog has heard the command roll_over.
Rover rolls over!
The dog has finished the command roll_over.


You can see that the **decorator** syntax lets us do all sorts of things much more conveniently. Let's add another that makes the dog bark 3 times:

In [10]:
# Define the decorator function
def add_dog_status(fun):
    def wrapper(*args, **kwargs): # Allow arguments to be passed to the original function
        print(f'The dog has heard the command {fun.__name__}.') # Add status before
        res = fun(*args, **kwargs) # Get any return values (there aren't any here)
        print(f'The dog has finished the command {fun.__name__}.') # Add status after
        return res # Return any return values
        
    return wrapper


def do_three_times(fun):
    def wrapper(*args, **kwargs): # Allow arguments to be passed to the original function
        for i in range(3):
            res = fun(*args, **kwargs) # Get any return values (there aren't any here)
        return res # Return the final return value (if there is one)
        
    return wrapper    

@do_three_times
def bark(name):
    print(f'{name} barks!')

@add_dog_status
def sit(name):
    print(f'{name} sits!')

@add_dog_status
def roll_over(name):
    print(f'{name} rolls over!')
    
def command_dog(name,command): # Make the dog called <name> do command <command>
    command(name)

command_dog("Rover",sit)
command_dog("Rover",bark)
command_dog("Rover",roll_over)

The dog has heard the command sit.
Rover sits!
The dog has finished the command sit.
Rover barks!
Rover barks!
Rover barks!
The dog has heard the command roll_over.
Rover rolls over!
The dog has finished the command roll_over.


### A Useful Example - Timing Functions

It's pretty common to want to be able to time how long a piece of code takes to run. We saw in a previous notebook how to do that using line-magic commands. Let's now do the same thing using a decorator. 

We recall our function that calculates $\pi$ using Monte Carlo sampling:

In [12]:
import numpy as np

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

monte_carlo_pi(1_000_000)

3.141976

Now let's add a decorator to time it:

In [23]:
import numpy as np
import time

def time_it(fun):
    def wrapper(*args, **kwargs):
        start = time.time()
        res = fun(*args, **kwargs)
        end = time.time()
        print(f'Function took {int(1000*(end-start))} milliseconds to execute.')
        
        return res
    
    return wrapper

@time_it
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

monte_carlo_pi(1_000_000)

Function took 1488 milliseconds to execute.


3.142296

### Adding decorators to other people's functions

In our timing example we effectively re-defined our function when we added a decorator to it. Often, we don't get to do that. Fortunately, there's another way to apply a decorator, which is by assigning our decorated function to another variable (i.e. it defines a separate function):

In [24]:
import numpy as np
import time

def time_it(fun):
    def wrapper(*args, **kwargs):
        start = time.time()
        res = fun(*args, **kwargs)
        end = time.time()
        print(f'Function took {int(1000*(end-start))} milliseconds to execute.')
        
        return res
    
    return wrapper

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

timed_monte_carlo_pi = time_it(monte_carlo_pi)

pi_estimate1 = monte_carlo_pi(1_000_000) # Use the ordinary function
print(f'The ordinary function gives pi = {pi_estimate1}')
pi_estimate2 = timed_monte_carlo_pi(1_000_000) # Use the timed definition
print(f'The timed function gives pi = {pi_estimate2}')

The ordinary function gives pi = 3.138556
Function took 1439 milliseconds to execute.
The timed function gives pi = 3.14066


We can adapt the code to make the timing more accurate by averaging over several executions of the function:

In [25]:
import numpy as np
import time

def time_it(fun):
    def wrapper(*args, **kwargs):
        start = time.time()
        for i in range(5):
            res = fun(*args, **kwargs)
            print(f'Iteration {i+1}') # Print which iteration we are doing
        end = time.time()
        print(f'Function took {int(1000/5*(end-start))} milliseconds to execute.')
        
        return res # Only return the last calculated value
    
    return wrapper

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

timed_monte_carlo_pi = time_it(monte_carlo_pi)

pi_estimate1 = monte_carlo_pi(1_000_000) # Use the ordinary function
print(f'The ordinary function gives pi = {pi_estimate1}')
pi_estimate2 = timed_monte_carlo_pi(1_000_000) # Use the timed definition
print(f'The timed function gives pi = {pi_estimate2}')

The ordinary function gives pi = 3.139928
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Function took 1464 milliseconds to execute.
The timed function gives pi = 3.145516


In fact, we can add an extra argument to choose how many execution times we wish to average over:

In [34]:
import numpy as np
import time

def time_it(fun,n):
    def wrapper(*args, **kwargs):
        start = time.time()
        for i in range(n):
            res = fun(*args, **kwargs)
            print(f'Iteration {i+1}') # Print which iteration we are doing
        end = time.time()
        print(f'Function took {int(1000/n*(end-start))} milliseconds to execute.')
        
        return res # Only return the last calculated value
    
    return wrapper

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = np.random.random()
        y = np.random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

timed_monte_carlo_pi = time_it(monte_carlo_pi,10)

pi_estimate1 = monte_carlo_pi(1_000_000) # Use the ordinary function
print(f'The ordinary function gives pi = {pi_estimate1}')
pi_estimate2 = timed_monte_carlo_pi(1_000_000) # Use the timed definition
print(f'The timed function gives pi = {pi_estimate2}')

The ordinary function gives pi = 3.138668
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Function took 1442 milliseconds to execute.
The timed function gives pi = 3.142784


### Example - Comparing the Speed of Random Number Generators

Let's compare 3 different methods of generating random numbers. Each method doesn't take very long to generate 1000 numbers, so let's compare each method using 100 executions of each. Here's the complete code, with added comments:

In [42]:
# Import required packages
import numpy as np
import time

# Define timing wrapper - hard-coded to do num_executions executions and then average
def time_it(fun):
    def wrapper(*args, **kwargs):
        num_executions = 100
        start = time.time()
        for _ in range(num_executions):
            res = fun(*args, **kwargs)
        end = time.time()
        print(f'Function took {int(1_000_000/num_executions*(end-start))} microseconds to execute.')
        
        return res # Only return the last calculated value
    
    return wrapper

# Define an LCG class so we can use it in our custom random number generator
class LCG:
    """
    A general linear congruential generator
    """
    def __init__(self, m, a, c):
        self.m = m
        self.a = a
        self.c = c
        self.seed = 0
        self.this_sample = self.seed # Set initial sequence value to be the seed
        # Can return the original seed value if we want to!
        
    def sample(self):
        # Generate the sample (between 0 and m)
        self.this_sample = (self.a * self.this_sample + self.c) % self.m
        # Return the sample (between 0 and 1)
        return self.this_sample/self.m
    
    # Allow the seed value to be set explicitly
    def set_seed(self, seed_val):
        self.seed = seed_val
        self.this_sample = self.seed

# We create an instance of the general LCG class, which generates (individual) samples using
# the a,m,c values used in the classic computing text 'Numerical Recipes'
numericalrecipes = LCG(2**32,1664525, 1013904223)

@time_it
def lcg_numpy_appended(n):
    random_set = np.array([]) # Initialise an empty Numpy array
    for _ in range(n): # Note we use a dummy variable _ because we're not using the index value
        # append values 1 by 1 (this is SLOW!) using the LCG instance for Numerical Recipes
        random_set = np.append(random_set, numericalrecipes.sample())
    return random_set

@time_it
def random_list_appended(n):
    random_set = [] # Initialise an empty Python list
    for _ in range(n): # Note we use a dummy variable _ because we're not using the index value
        # append values 1 by 1 (this is SLOW!)
        random_set.append(np.random.random())
    return random_set

@time_it
def random_numpy_appended(n):
    random_set = np.array([]) # Initialise an empty Numpy array
    for _ in range(n): # Note we use a dummy variable _ because we're not using the index value
        # append values 1 by 1 (this is SLOW!)
        random_set = np.append(random_set, np.random.random())
    return random_set

@time_it
def random_numpy_allatonce(n):
    return np.random.random(n) # This is the best way to generate random numbers fast
        
# Time the different random number generators
sample1 = lcg_numpy_appended(10_000)
sample2 = random_list_appended(10_000)
sample3 = random_numpy_appended(10_000)
sample4 = random_numpy_allatonce(10_000)

print('Finished!')

Function took 122434 microseconds to execute.
Function took 7179 microseconds to execute.
Function took 120392 microseconds to execute.
Function took 118 microseconds to execute.
Finished!


You can see that there is a large difference in speed between the different methods. The quickest one is the in-built **numpy.random.random(n)**, which utilises compiled code to execute. Whilst in this case the differences are not that significant - we are only creating one sample set - you will hopefully appreciate that timing differences can become very important if we are executing the same function over and over again.

Timing different implementations of the same functions is very important in optimising large programs.